In [1]:
%run project_functions.ipynb

Using TensorFlow backend.


In [2]:
file_name = 'FB.json'
csv_name = 'FB.csv'
T = 3
capital = 90000

In [3]:
#Features
company = stock_twits_text_parser(file_name)
company_metrics = stock_twits_metrics(company, T, file_name)

#Labels
company_return = return_over_period_T(csv_name,3)

#Number of Trees:
n = 400

#Cleaned Data
agg_data = feature_selector(company_metrics, company_return, 'close_return', 25, n)
company_data = split_data(agg_data, 0.7, 'all', 'close_return', 'bin')

#Split
X_train = company_data['X_train']
Y_train = company_data['Y_train']
X_test = company_data['X_test']
Y_test = company_data['Y_test']

In [4]:
#Fit Model
clf_rf = RandomForestClassifier().fit(X_train, Y_train)

#Predict
pred_test = clf_rf.predict(X_test)

# Trading Scenario Using Predictions from Random Forrest Classifier

In [5]:
trader_single_stock(pred_test, T, 'close', company_return, 90000, len(X_train))

60000.0
40000.0
26666.666666666664
48211.921565602555
51607.56806999476
47871.19859677851
40642.852329868794
43227.901584577354
46104.67747841343
46837.28421157908
45072.069204631414
44864.293613140035
45942.207359355285
46077.40578145749
45692.77834089364
45451.2684710615
45536.50967966202
45418.82897044688
45087.20289737349
45547.574230499034
45423.589763635944
45189.76073596986
44981.262345257324
45175.22405129616
45003.69045285107
45402.91878028026
45177.065606245116
45208.03711083484
45182.13721795147
45276.606515895415
45360.69738359522
45501.9476762837
45455.94571004433
45396.47074612887
45415.50965774499
45470.23750078211
45520.80612188759
45488.239320795714
45494.78560776745
45557.590632448104
45920.429458993654
46014.10592711808
45699.31289546175
45695.575547547465
45688.116998984275
45356.83996516222
44880.90587042815
45604.265280158266
45940.07833453081
46017.068339939695
45988.899409933096
45939.924279392515
45979.54071528309
46215.42593667286
46244.59784293425
46107.54640

97989.99588015742

In [9]:
company_return

,Date,OPEN,HIGH,LOW,VOLUME,CLOSE,open_return,high_return,low_return,close_return
0,2015-07-23,96.959999,97.449997,94.809998,29418800,95.440002,-0.021865,-0.019395,-0.015821,-0.001572
1,2015-07-24,97.349998,97.760002,95.879997,33444900,96.949997,-0.010580,-0.004910,-0.011890,0.000413
2,2015-07-27,96.580002,96.610001,93.830002,38585400,94.169998,-0.017291,-0.008281,-0.021635,0.011044
3,2015-07-28,94.839996,95.559998,93.309998,35236000,95.290001,0.001160,0.008058,0.006859,-0.013433
4,2015-07-29,96.320000,97.279999,94.739998,64648300,96.989998,-0.028966,-0.022615,-0.020477,-0.029384
...,...,...,...,...,...,...,...,...,...,...
338,2016-11-22,122.400002,122.980003,120.900002,25992700,121.470001,-0.018627,-0.010490,-0.008933,-0.008726
339,2016-11-23,121.230003,121.309998,119.940002,15592400,120.839996,-0.005444,0.006512,0.003835,0.000248
340,2016-11-25,121.010002,121.139999,120.070000,8638400,120.379997,-0.005702,0.005366,-0.017656,-0.016282
341,2016-11-28,120.120003,121.690002,119.820000,18073300,120.410004,-0.014486,-0.026625,-0.048573,-0.044099


In [10]:
pred_test

array([ 1, -1,  1, -1, -1, -1, -1,  1,  1,  1, -1, -1, -1, -1, -1, -1,  1,
       -1,  1,  1,  1, -1,  1, -1, -1, -1, -1, -1, -1, -1,  1,  1, -1,  1,
       -1, -1,  1,  1,  1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1, -1,  1,
        1, -1,  1, -1, -1, -1, -1,  1, -1, -1, -1,  1, -1, -1,  1, -1, -1,
       -1, -1, -1,  1, -1, -1,  1, -1, -1,  1, -1, -1,  1, -1,  1, -1, -1,
       -1,  1,  1,  1, -1, -1, -1, -1,  1,  1, -1, -1,  1,  1, -1, -1])

In [6]:
# Trading Scenario Using Predictions from Random Forrest Classifier


In [7]:
trader_single_stock(Y_test.values, T, 'close', company_return, 90000, len(X_train))

60000.0
40000.0
26666.666666666664
48211.921565602555
52672.367047178624
48581.06458156775
41435.28331991567
43756.18891127527
46813.52595375334
47548.603719984734
45816.26721690504
45541.49468746383
46640.17336621147
47106.6302896282
46724.94271824871
46365.634795262216
46534.810378443115
47028.1057943589
47357.62712679563
47372.7993186281
47214.51677614126
47390.969540576974
47561.03506888833
47503.68891815491
47669.729709876
47930.43991993349
47895.58596518191
47798.16697523585
47800.071088268625
47865.56491799532
47999.899872524526
48135.800841352044
48088.00380401857
48014.17970303178
48042.18465388846
48100.80530410608
48155.074636834215
48117.499936265755
48124.96510809848
48192.00638597373
48576.50301987874
48674.983413774746
48639.595897112056
48536.41843020682
48635.026372824315
49111.79628734184
49580.526716466964
49712.381550229955
49680.9829432783
49785.12547404888
50104.185172898295
50059.957958478524
49976.41982586493
50154.321954765066
50251.65772909424
50224.4844307796

115316.43397943024